In [10]:
#import libraries
import pandas as pd
import pymongo
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy import text

## Create Connection to postgres

In [13]:
conn = psycopg2.connect(database="postgres", user='postgres', password='postgres', host='127.0.0.1', port='5432')
conn.autocommit = True

OperationalError: FATAL:  password authentication failed for user "postgres"


### Create Database in postgres

In [44]:
cursor = conn.cursor()

sql = '''CREATE database beer_db''';

cursor.execute(sql)
print("Database created successfully........")

#Closing the connection
conn.close()

Database created successfully........


## Store CSV into DataFrame

In [3]:
csv_file = "../ETL-Project/data/archive/beers.csv"
beers_df = pd.read_csv(csv_file)
beers_df.head()

,Unnamed: 0,abv,ibu,id,name,style,brewery_id,ounces
0,0,0.050,NaN,1436,Pub Beer,American Pale Lager,408,12.0
1,1,0.066,NaN,2265,Devil's Cup,American Pale Ale (APA),177,12.0
2,2,0.071,NaN,2264,Rise of the Phoenix,American IPA,177,12.0
3,3,0.090,NaN,2263,Sinister,American Double / Imperial IPA,177,12.0
4,4,0.075,NaN,2262,Sex and Candy,American IPA,177,12.0


In [4]:
csv_file = "../ETL-Project/data/archive/breweries.csv"
breweries_df = pd.read_csv(csv_file)
breweries_df.head()

,Unnamed: 0,name,city,state
0,0,NorthGate Brewing,Minneapolis,MN
1,1,Against the Grain Brewery,Louisville,KY
2,2,Jack's Abby Craft Lagers,Framingham,MA
3,3,Mike Hess Brewing Company,San Diego,CA
4,4,Fort Point Beer Company,San Francisco,CA


In [5]:
breweries_df.rename(columns = {'Unnamed: 0':'brewery_id', 'name':'brewery_name'}, inplace = True) 
breweries_df.head()


,brewery_id,brewery_name,city,state
0,0,NorthGate Brewing,Minneapolis,MN
1,1,Against the Grain Brewery,Louisville,KY
2,2,Jack's Abby Craft Lagers,Framingham,MA
3,3,Mike Hess Brewing Company,San Diego,CA
4,4,Fort Point Beer Company,San Francisco,CA


In [6]:
breweries_df.set_index("brewery_id")

,brewery_name,city,state
brewery_id,,,
0,NorthGate Brewing,Minneapolis,MN
1,Against the Grain Brewery,Louisville,KY
2,Jack's Abby Craft Lagers,Framingham,MA
3,Mike Hess Brewing Company,San Diego,CA
4,Fort Point Beer Company,San Francisco,CA
...,...,...,...
553,Covington Brewhouse,Covington,LA
554,Dave's Brewfarm,Wilson,WI
555,Ukiah Brewing Company,Ukiah,CA


In [8]:
breweries_df.state = breweries_df.state.str.lstrip()
breweries_df

,brewery_id,brewery_name,city,state
0,0,NorthGate Brewing,Minneapolis,MN
1,1,Against the Grain Brewery,Louisville,KY
2,2,Jack's Abby Craft Lagers,Framingham,MA
3,3,Mike Hess Brewing Company,San Diego,CA
4,4,Fort Point Beer Company,San Francisco,CA
...,...,...,...,...
553,553,Covington Brewhouse,Covington,LA
554,554,Dave's Brewfarm,Wilson,WI
555,555,Ukiah Brewing Company,Ukiah,CA
556,556,Butternuts Beer and Ale,Garrattsville,NY


In [49]:
#drop columns in beer_df
beers_df = beers_df.drop(columns=['Unnamed: 0', 'ibu'])
#beers_df.head()

In [50]:
beers_df.rename(columns = {'id':'beer_id', 'name':'beer_name'}, inplace = True) 

In [51]:
beers_df.set_index("beer_id")

,abv,beer_name,style,brewery_id,ounces
beer_id,,,,,
1436,0.050,Pub Beer,American Pale Lager,408,12.0
2265,0.066,Devil's Cup,American Pale Ale (APA),177,12.0
2264,0.071,Rise of the Phoenix,American IPA,177,12.0
2263,0.090,Sinister,American Double / Imperial IPA,177,12.0
2262,0.075,Sex and Candy,American IPA,177,12.0
...,...,...,...,...,...
928,0.067,Belgorado,Belgian IPA,424,12.0
807,0.052,Rail Yard Ale,American Amber / Red Ale,424,12.0
620,0.055,B3K Black Lager,Schwarzbier,424,12.0


## Build Schema in SQL

In [34]:
conn = psycopg2.connect(database="beer_db", user='postgres', password='postgres', host='127.0.0.1', port='5432')
conn.autocommit = True
cursor = conn.cursor()

In [53]:
schema_sql = open('Schema Build.sql','r')
cursor.execute(schema_sql.read())
conn.close()

## Create connection for Load operation

In [13]:
connection_string = "postgres:postgres@localhost:5432/beer_db"
engine = create_engine(f'postgresql://{connection_string}')

In [14]:
# Confirm tables
engine.table_names()

['brewery', 'beers']

### Load the DF into Postgres

In [56]:
breweries_df.to_sql(name='brewery', con=engine, if_exists='append', index=False)

In [57]:
beers_df.to_sql(name='beers', con=engine, if_exists='append', index=False)

### Dispose of the Connection to the Database

In [58]:
engine.dispose()

In [35]:
sql = conn.cursor()

In [77]:
sql.execute("SELECT bw.brewery_name, bw.state, br.beer_name FROM BEERS br INNER JOIN BREWERY bw ON br.BREWERY_ID = bw.BREWERY_ID WHERE bw.state = ' TX'")
result_data = sql.fetchall()

In [78]:
col_names = []
for c in sql.description:
    col_names.append(c[0])

In [79]:
df = pd.DataFrame(result_data, columns=col_names)
df.head()

,brewery_name,state,beer_name
0,Armadillo Ale Works,TX,Quakertown Stout
1,Armadillo Ale Works,TX,Greenbelt Farmhouse Ale
2,Austin Beerworks,TX,Heavy Machinery IPA Series #1: Heavy Fist
3,Austin Beerworks,TX,Fire Eagle IPA
4,Austin Beerworks,TX,Peacemaker


In [ ]:
conn.close()